<a href="https://colab.research.google.com/github/aquapathos/MLLesson/blob/master/ML08%20%E4%B8%80%E8%88%AC%E7%89%A9%E4%BD%93%E8%AA%8D%E8%AD%98/AIChallenge/003_%E7%94%BB%E5%83%8F%E3%81%AE%E6%B0%B4%E5%A2%97%E3%81%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. 準備
## 0.1 ハードウェアアクセラレータの設定

1. [「AIChallenge001-画像の収集」を済ませておく](https://github.com/aquapathos/BasicAI/blob/master/AIChallenge001_%E7%94%BB%E5%83%8F%E3%81%AE%E5%8F%8E%E9%9B%86.ipynb)

2. [「ランタイム」メニューを開く]()
3. [「ランタイムのタイプを変更」をクリック]()
4. [[「ハードウェアアクセラレータ」で **GPU** を選択し，「保存」]()


# 1. Google Drive をマウント
AIChallenge001 で保存した画像データを使うために Google Drive に接続します。

1. [次のセルを実行する]()
2. [リンクと入力フィールドが表示されるのでリンクをクリック]()
3. [アクセスリクエストを許可するとアクセス用のコードが表示されるのでコピー]()
4. [Enter your authorization code: フィールドに貼り付けて Enter]()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# ２．　画像の読み込みと表示
##  2.1 関数定義
001 で使ったプログラムと同じです。　[次のセルを実行]() 



In [ ]:
import pickle
import os,math
import numpy as np
import matplotlib.pyplot as plt
    
# pickle 形式で保存された画像データの読み込み
def loadCategoryImages(fname, folder = "."):
    f = open(folder+"/"+fname,'rb')
    cat = pickle.load(f)
    f.close
    return cat

# 画像データの表示
# start番からnpic枚表示する関数を定義
plt.rcParams['figure.figsize'] = (12.0, 7.0)
def showimg(images, start = 0, npic = 48):
    n = npic if len(images) >= start+npic else len(images) - start
    plt.figure(figsize=(8,7.5*(math.ceil(n/8))/6),dpi=150)
    i = 0
    while True:
        if i < n :  
            plt.subplot((n-1)//8+1,8,i+1)
            plt.xticks([])
            plt.yticks([])
            plt.imshow(images[start+i][:,:,::-1])
            plt.title("{}".format(start+i))
            i += 1
        else:
            break

## 2.2 画像データの読み込みと内容確認

先に収集した画像データを読み込み、一部を表示して内容が同じか確認しておきます。　[次のセルを実行]() 

In [ ]:
GFOLDER = "drive/My Drive/LDATA"  # データ保存用のフォルダ名
c0img = loadCategoryImages("ネコ.pkl", folder=GFOLDER)
c1img = loadCategoryImages("イヌ.pkl", folder=GFOLDER)
showimg(c0img,0,16)
showimg(c1img,0,16)

# データの水増し
**[ImageDataGenerator](https://keras.io/ja/preprocessing/image/)** を用いてデータを水増しする。

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import shutil
from glob import glob

def dataInflation1(inimg,num):

    datagen = ImageDataGenerator(
            rotation_range=0, # 画像をランダムに回転する
            width_shift_range=0.2, # ランダムに水平シフト
            height_shift_range=0.2, # ランダムに垂直シフト
            shear_range=5, # 剪断
            zoom_range=0.2, # ランダムにズームする範囲
            horizontal_flip=True, # 水平方向に入力をランダムに反転
            vertical_flip=True, # 垂直方向に入力をランダムに反転
            rescale=1.0 / 255, # 与えられた値をデータに積算する
            )
    
    # 出力先ディレクトリを作成
    tmpdir = "tmp/tmp"
    os.makedirs(tmpdir,exist_ok=True)
    inimg = inimg.reshape((1,) + inimg.shape)  # 次元を４次元に（ImageDataGeneratorの仕様)

    # 種となる画像から num+1 個のバリエーションを生成 
    g = datagen.flow(inimg, batch_size=1, save_to_dir=tmpdir, save_prefix='img', save_format='png')
    for i in range(num+1): # すべて成功すればnum-1個で良いのだが、稀に生成に失敗するので２つ余分に作っておく
        batch = g.next()

    pathes = glob(os.path.join(tmpdir, "*.png"))[:num-1]
    outimgs = [inimg.reshape(inimg.shape[1:])]
    for files in pathes:
        img = image.load_img(files)
        outimgs.append(np.array(img))
    shutil.rmtree(tmpdir)
    return np.array(outimgs)

def dataInflation(X,num):
    Xs = []
    for img in X:
        Xs.extend(dataInflation1(img,num=num))
    return Xs

### 動作確認


In [ ]:
IMG = dataInflation(c0img,4)
showimg(IMG,48)

# 3. 学習と認識の実験

## 3.1 学習モデル

水増しなしの時と同じなので説明省略

## 3.2 学習データの作成

画像データを訓練用とテスト用に分割し，学習用のデータをXtrainとXtestに分割します．

[次のセルを実行]() 

In [ ]:
#  変数の初期化

INFLATION = 24
def make_dataset(catalist):
    # catlist で与えられた画像データのリストから訓練用とテスト用のデータを作成し，
    # Xtrain (訓練用画像), ytrain（訓練例の正解)，Xtrain (テスト用画像), ytrain（テスト画像に対する)，NDATA（1カテゴリ当たりのデータ数）
    # 学習に用いるデータ数を、一番データ数の少ないカテゴリのデータ数に合わせる
    mindata = np.inf # 十分大きな数
    for cat in catalist:
        if len(cat) < mindata:
            mindata = len(cat)
    # ここに到達した時点で、mindata にはもっともデータ数の少ないカテゴリのデータ数が入っている
    NDATA = mindata
    threer = mindata%3
    NDATA = mindata - threer  #  個数を３の倍数となるよう調整

    Xtrain,Xtest = [],[]  # 入力画像のリスト
    ytrain,ytest = [],[]  #  ラベルのリスト
    for cimgs in catalist:
        Xtrain = Xtrain + cimgs[0:int(2*NDATA/3)]
        Xtest = Xtest + cimgs[int(2*NDATA/3):NDATA]
    # 訓練用データXtrainをINFLATION倍に水増し
    Xtrain = dataInflation(Xtrain,INFLATION)
    for label in range(len(catalist)):
        ytrain = ytrain + [label]*int(2*NDATA/3*INFLATION)
        ytest = ytest + [label]*int(NDATA/3)

    return Xtrain,ytrain,Xtest,ytest,NDATA

次のプログラムで，**CATALIST = [c1img,c2img]** の部分で認識対象となるカテゴリの画像データを指定する．カテゴリは２つである必要はなく，例えば，４つならば，

**CATALIST = [c1img,c2img,c3img,c4img]**   

等としてください．　[次のセルを実行]() 

In [ ]:
CATALIST = [c0img,c1img] # c0img:ネコ，c1img：イヌ
Xtrain,ytrain,Xtest,ytest,NDATA = make_dataset(CATALIST) 

# データ数の確認
ntrain = len(Xtrain)
ntest  = len(Xtest)
print("カテゴリ数 {}   1カテゴリ当たりのデータ数 {}".format(len(CATALIST),NDATA))
print("訓練用 {} テスト用 {}  　データ総数 {}".format(ntrain,ntest,ntrain+ntest))

# 正解データの確認
print(ytest)

# model1  全結合２層ネットワーク

## モデル定義
[以下のセルを順に実行していってください．]() 

In [ ]:
# tensorflow2.x を前提としている。1.x の場合、tensorflow.keras を ただの keras に変える。
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers  import Input, Activation, Dropout, Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger,TensorBoard

SIZE = 128

# 変数の宣言
CLASSES= len(CATALIST)  #  カテゴリ数　ここでは　2　
DATASIZE = SIZE *  SIZE * 3

# Model 1
model1 = Sequential()
model1.add(Dense(CLASSES, activation='softmax', input_shape=(DATASIZE,)))

model1.summary()

from IPython.display import Image, display_png
#学習モデル図の作成
plot_model(model1, to_file='model1.png')
display_png(Image('model1.png'))

## ニューラルネットワークの設定と学習の実行

In [ ]:
model = model1

model.compile(loss='sparse_categorical_crossentropy',
              optimizer = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])

# データを 浮動小数点数に変換し、[0,255] → [0,1] に変換
X = (np.array(Xtrain).reshape(len(Xtrain),DATASIZE).astype('float32'))/255
y = np.array(ytrain).astype('float32')
es = EarlyStopping(monitor='loss', patience=5)   #  訓練用データのロスが改善されなくなったら5エポック後に停止
tb_cb = TensorBoard(log_dir='tblog', histogram_freq=1, write_graph=True)
csv_logger = CSVLogger('training.log')
hist = model.fit(X , y,
                 epochs=100,
                 verbose=1,
                 callbacks=[es, csv_logger])

### 学習過程のグラフ化

In [ ]:
from matplotlib import cm
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
acc = hist.history['accuracy']
loss = hist.history['loss']
ax1.plot(range(1, len(loss)+1), loss,color=cm.Set1.colors[0],label='loss') # 誤差
ax2.plot(range(1, len(acc)+1), acc,color=cm.Set1.colors[1],label='accuracy') #  正解率
ax1.set_ylabel('Loss')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')
ax1.legend()
ax2.legend()
plt.show()

## 訓練データに対する識別結果

In [ ]:
import pandas as pd

catnamelist = ['ネコ','イヌ']

# 訓練データに対する識別結果
def recognitionResult(Xdata, ydata, catnamelist=catnamelist):
    ndata = len(Xdata) # データ数
    predictT = model.predict(Xdata)
    predictT = [np.argmax(n1)  for n1 in predictT]
    NCAT = len(CATALIST) # カテゴリ数
    ct1 = np.zeros((NCAT,NCAT),np.uint16) # 認識結果集計表
    Error = []
    for i in range(ndata):
        ct1[ydata[i],predictT[i]] += 1
        if ydata[i] != predictT[i]:
            Error.append([i,ydata[i],predictT[i]])
    print("誤認識データ（[データ番号, 正解, 認識結果]）\n {0} \n　正解率={1:5.1f}　誤り率＝{2:5.1f} %\n".format(Error,100*(ndata-len(Error))/ndata,100*len(Error)/ndata))
    print("正解カテゴリに対する認識結果と正解率")
    crossT1 = pd.concat([pd.DataFrame(catnamelist,columns=['正解カテゴリ']),pd.DataFrame(ct1,columns=catnamelist)],axis=1)
    crossT1 = pd.concat([crossT1,pd.DataFrame([np.round(1000*crossT1[cat][i]/ndata*NCAT)/10 for i,cat in enumerate(catnamelist)],columns=['正解率'])],axis=1).set_index('正解カテゴリ')

    return Error, crossT1

TrainError, crossTable = recognitionResult(Xdata=X, ydata=ytrain, catnamelist=catnamelist)
crossTable

In [ ]:
# 認識間違いの表示
plt.rcParams['figure.figsize'] = (12.0, 7.0)
def showEimg(errlist, images):
    last = len(errlist) if len(errlist) < 48 else 48
    if last == 0 : return
    plt.figure(figsize=(8,7.5*(math.ceil(last/8))/6),dpi=150)
    for i in range(last):
            plt.subplot((last-1)//8+1,8,i+1)
            plt.xticks([])
            plt.yticks([])
            plt.imshow(images[errlist[i][0]][:,:,::-1])
            plt.title("{}→{}".format(errlist[i][1],errlist[i][2]))

showEimg(TrainError,Xtrain)
# 0:ネコ、1:イヌ

## テストデータに対する識別結果

In [ ]:
# テストデータでの識別結果
X = (np.array(Xtest).reshape(len(Xtest),DATASIZE).astype('float32'))/255
TestError, crossTable = recognitionResult(Xdata=X, ydata=ytest, catnamelist=catnamelist)
crossTable

In [ ]:
# 誤認識したデータの表示
showEimg(TestError,Xtest)
# 0:ネコ、1:イヌ

# model 2　　３層全結合バックプロパゲーションネットワーク（中間層1024ノード）

## モデル定義

In [ ]:
# Model 2

model2 = Sequential()
model2.add(Dense(1024,activation='relu',input_shape=(DATASIZE,)))
# model2.add(Dropout(rate=0.5))
model2.add(Dense(CLASSES, activation='softmax'))

model2.summary()

#学習モデル図の作成
plot_model(model2, to_file='model2.png')
display_png(Image('model2.png'))

## ニューラルネットワークの設定と学習の実行

In [ ]:
model = model2

model.compile(loss='sparse_categorical_crossentropy',
              optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])

# データを 浮動小数点数に変換し、[0,255] → [0,1] に変換
X = (np.array(Xtrain).reshape(len(Xtrain),DATASIZE).astype('float32'))/255
y = np.array(ytrain).astype('float32')
es = EarlyStopping(monitor='loss', patience=5)   #  訓練用データのロスが改善されなくなったら5エポック後に停止
tb_cb = TensorBoard(log_dir='tblog', histogram_freq=1, write_graph=True)
csv_logger = CSVLogger('training.log')
hist = model.fit(X , y,
                 epochs=200,
                 verbose=1,
                 callbacks=[es, csv_logger])

### 学習過程のグラフ化

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
acc = hist.history['accuracy']
loss = hist.history['loss']
ax1.plot(range(1, len(loss)+1), loss,color=cm.Set1.colors[0],label='loss')
ax2.plot(range(1, len(acc)+1), acc,color=cm.Set1.colors[1],label='accuracy')
ax1.set_ylabel('Loss')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')
ax1.legend()
ax2.legend()
plt.show()

## 訓練データに対する識別結果

In [ ]:
# 訓練データに対する識別結果
TrainError, crossTable = recognitionResult(Xdata=X, ydata=ytrain, catnamelist=catnamelist)
crossTable

In [ ]:
# 誤認識した画像を表示
showEimg(TrainError,Xtrain)
# 0:ネコ、1:イヌ

## テストデータに対する識別結果

In [ ]:
# テストデータでの識別結果
X = (np.array(Xtest).reshape(len(Xtest),DATASIZE).astype('float32'))/255
TestError, crossTable = recognitionResult(Xdata=X, ydata=ytest, catnamelist=catnamelist)
crossTable

In [ ]:
# 誤認識したデータの表示
showEimg(TestError,Xtest)
# 0:ネコ、1:イヌ

# model 3  CNN(畳み込みネットワーク）

## モデル定義

In [ ]:
# Model ３

model3 = Sequential()
model3.add(Conv2D(32, kernel_size=(3, 3),input_shape=(SIZE, SIZE,3), activation='relu',padding='same'))
model3.add(Conv2D(64, kernel_size=(3, 3), activation='relu',padding='same'))
model3.add(MaxPooling2D(pool_size=(2, 2)))  # 64 x 64 x 64
model3.add(Conv2D(128, kernel_size=(3, 3), activation='relu',padding='same'))
model3.add(MaxPooling2D(pool_size=(2, 2)))  #  32 x 32 x 128
model3.add(Conv2D(64, kernel_size=(3, 3), activation='relu',padding='same'))
model3.add(Flatten())
model3.add(Dense(1024, activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(64, activation='relu'))
model3.add(Dense(CLASSES, activation='softmax'))

model3.summary()
model3.compile(loss='categorical_crossentropy',
              optimizer = optimizers.Adam(),
              metrics=['accuracy'])

#学習モデル図の作成
plot_model(model3, to_file='model3.png')
display_png(Image('model3.png'))


## ニューラルネットワークの設定と学習の実行

In [ ]:
model = model3

model.compile(loss='sparse_categorical_crossentropy',
              optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])

# データを 浮動小数点数に変換し、[0,255] → [0,1] に変換
X = (np.array(Xtrain).astype('float32'))/255
y = np.array(ytrain).astype('float32')
es = EarlyStopping(monitor='loss', patience=3)   #  訓練用データのロスが改善されなくなったら3エポック後に停止
tb_cb = TensorBoard(log_dir='tblog', histogram_freq=1, write_graph=True)
csv_logger = CSVLogger('training.log')
hist = model.fit(X , y,
                 epochs=100,
                 verbose=1,
                 callbacks=[es, csv_logger])

### 学習過程のグラフ化

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
acc = hist.history['accuracy']
loss = hist.history['loss']
ax1.plot(range(1, len(loss)+1), loss,color=cm.Set1.colors[0],label='loss')
ax2.plot(range(1, len(acc)+1), acc,color=cm.Set1.colors[1],label='accuracy')
ax1.set_ylabel('Loss')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')
ax1.legend()
ax2.legend()
plt.show()

## 訓練データに対する識別結果

In [ ]:
# 訓練データに対する識別結果
TrainError, crossTable = recognitionResult(Xdata=X, ydata=ytrain, catnamelist=catnamelist)
crossTable

In [ ]:
# 誤認識した画像を表示
showEimg(TrainError,Xtrain)
# 0:ネコ、1:イヌ

## テストデータに対する識別結果

In [ ]:
# テストデータでの識別結果
Xte = (np.array(Xtest).astype('float32'))/255
TestError, crossTable = recognitionResult(Xdata=Xte, ydata=ytest, catnamelist=catnamelist)
crossTable

In [ ]:
showEimg(TestError[:96],Xtest)
# 0:ネコ、1:イヌ

# ネコとカメ

イヌの代わりにカメで同じ実験をやってみましょう．

In [ ]:
# c0img = loadCategoryImages("ネコ.pkl", folder=GFOLDER)
c2img = loadCategoryImages("カメ.pkl", folder=GFOLDER)
showimg(c0img,0,16)
showimg(c2img,0,16)

In [ ]:
catnamelist = ['ネコ','カメ']
CATALIST=[c0img,c2img] # c0img:ネコ, c2img:カメ

# 訓練用とテスト用に分割
Xtrain,ytrain,Xtest,ytest,NDATA = make_dataset(CATALIST) 

# データ数の確認
ntrain = len(Xtrain)
ntest  = len(Xtest)
print("カテゴリ数 {}   1カテゴリ当たりのデータ数 {}".format(len(CATALIST),NDATA))
print("訓練用 {} テスト用 {}  　データ総数 {}".format(ntrain,ntest,ntrain+ntest))

# 正解データの確認 (ytrainは多すぎるので ytest のみ表示)
print(ytest)

## model1

In [ ]:
# 変数の宣言
CLASSES= len(CATALIST)  #  カテゴリ数　ここでは　2　
DATASIZE = SIZE *  SIZE * 3

# Model 1
model1 = Sequential()
model1.add(Dense(CLASSES, activation='softmax', input_shape=(DATASIZE,)))

model = model1
model.compile(loss='sparse_categorical_crossentropy',
              optimizer = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])

# データを 浮動小数点数に変換し、[0,255] → [0,1] に変換
X = (np.array(Xtrain).reshape(len(Xtrain),DATASIZE).astype('float32'))/255
y = np.array(ytrain).astype('float32')
es = EarlyStopping(monitor='loss', patience=5)   #  訓練用データのロスが改善されなくなったら5エポック後に停止
tb_cb = TensorBoard(log_dir='tblog', histogram_freq=1, write_graph=True)
csv_logger = CSVLogger('training.log')
hist = model.fit(X , y,
                 epochs=100,
                 verbose=1,
                 callbacks=[es, csv_logger])

In [ ]:
from matplotlib import cm
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
acc = hist.history['accuracy']
loss = hist.history['loss']
ax1.plot(range(1, len(loss)+1), loss,color=cm.Set1.colors[0],label='loss') # 誤差
ax2.plot(range(1, len(acc)+1), acc,color=cm.Set1.colors[1],label='accuracy') #  正解率
ax1.set_ylabel('Loss')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')
ax1.legend()
ax2.legend()
plt.show()

## 訓練データに対する識別実験結果

In [ ]:
TrainError, crossTable = recognitionResult(Xdata=X, ydata=ytrain, catnamelist=catnamelist)
crossTable

In [ ]:
showEimg(TrainError,Xtrain)
# 0:ネコ、1:カメ

## テストデータに対する識別実験結果

In [ ]:
# テストデータでの識別結果
X = (np.array(Xtest).reshape(len(Xtest),DATASIZE).astype('float32'))/255
TestError, crossTable = recognitionResult(Xdata=X, ydata=ytest, catnamelist=catnamelist)
crossTable

In [ ]:
# 誤認識したデータの表示
showEimg(TestError,Xtest)
# 0:ネコ、1:カメ

## model2

In [ ]:
# Model 2

model2 = Sequential()
model2.add(Dense(1024,activation='relu',input_shape=(DATASIZE,)))
# model2.add(Dropout(rate=0.5))
model2.add(Dense(CLASSES, activation='softmax'))

model = model2
model.compile(loss='sparse_categorical_crossentropy',
              optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])

# データを 浮動小数点数に変換し、[0,255] → [0,1] に変換
X = (np.array(Xtrain).reshape(len(Xtrain),DATASIZE).astype('float32'))/255
y = np.array(ytrain).astype('float32')
es = EarlyStopping(monitor='loss', patience=5)   #  訓練用データのロスが改善されなくなったら2エポック後に停止
tb_cb = TensorBoard(log_dir='tblog', histogram_freq=1, write_graph=True)
csv_logger = CSVLogger('training.log')
hist = model.fit(X , y,
                 epochs=100,
                 verbose=1,
                 callbacks=[es, csv_logger])

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
acc = hist.history['accuracy']
loss = hist.history['loss']
ax1.plot(range(1, len(loss)+1), loss,color=cm.Set1.colors[0],label='loss')
ax2.plot(range(1, len(acc)+1), acc,color=cm.Set1.colors[1],label='accuracy')
ax1.set_ylabel('Loss')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')
ax1.legend()
ax2.legend()
plt.show()

## 訓練データに対する識別実験結果

In [ ]:
# 訓練データに対する識別結果
TrainError, crossTable = recognitionResult(Xdata=X, ydata=ytrain, catnamelist=catnamelist)
crossTable

In [ ]:
# 誤認識した画像を表示
showEimg(TrainError,Xtrain)
# 0:ネコ、1:カメ

## テストデータに対する識別実験結果

In [ ]:
# テストデータでの識別結果
X = (np.array(Xtest).reshape(len(Xtest),DATASIZE).astype('float32'))/255
TestError, crossTable = recognitionResult(Xdata=X, ydata=ytest, catnamelist=catnamelist)
crossTable

In [ ]:
# 誤認識したデータの表示
showEimg(TestError,Xtest)
# 0:ネコ、1:カメ

# model3

In [ ]:
# Model ３

model3 = Sequential()
model3.add(Conv2D(32, kernel_size=(3, 3),input_shape=(SIZE, SIZE,3), activation='relu',padding='same'))
model3.add(Conv2D(64, kernel_size=(3, 3), activation='relu',padding='same'))
model3.add(MaxPooling2D(pool_size=(2, 2)))  # 64 x 64 x 64
model3.add(Conv2D(128, kernel_size=(3, 3), activation='relu',padding='same'))
model3.add(MaxPooling2D(pool_size=(2, 2)))  #  32 x 32 x 128
model3.add(Conv2D(64, kernel_size=(3, 3), activation='relu',padding='same'))
model3.add(Flatten())
model3.add(Dense(1024, activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(64, activation='relu'))
model3.add(Dense(CLASSES, activation='softmax'))

model3.compile(loss='categorical_crossentropy',
              optimizer = optimizers.Adam(),
              metrics=['accuracy'])

model = model3
model.compile(loss='sparse_categorical_crossentropy',
              optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])

# データを 浮動小数点数に変換し、[0,255] → [0,1] に変換
X = (np.array(Xtrain).astype('float32'))/255
y = np.array(ytrain).astype('float32')
es = EarlyStopping(monitor='loss', patience=3)   #  訓練用データのロスが改善されなくなったら3エポック後に停止
tb_cb = TensorBoard(log_dir='tblog', histogram_freq=1, write_graph=True)
csv_logger = CSVLogger('training.log')
hist = model.fit(X , y,
                 epochs=100,
                 verbose=1,
                 callbacks=[es, csv_logger])

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
acc = hist.history['accuracy']
loss = hist.history['loss']
ax1.plot(range(1, len(loss)+1), loss,color=cm.Set1.colors[0],label='loss')
ax2.plot(range(1, len(acc)+1), acc,color=cm.Set1.colors[1],label='accuracy')
ax1.set_ylabel('Loss')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')
ax1.legend()
ax2.legend()
plt.show()

# 訓練データに対する識別実験結果

In [ ]:
# 訓練データに対する識別結果
TrainError, crossTable = recognitionResult(Xdata=X, ydata=ytrain, catnamelist=catnamelist)
crossTable

In [ ]:
# 誤認識した画像を表示
showEimg(TrainError,Xtrain)
# 0:ネコ、1:カメ

## テストデータに対する識別実験結果

In [ ]:
# テストデータでの識別結果
Xte = (np.array(Xtest).astype('float32'))/255
TestError, crossTable = recognitionResult(Xdata=Xte, ydata=ytest, catnamelist=catnamelist)
crossTable

In [ ]:
showEimg(TestError,Xtest)
# 0:ネコ、1:カメ

# ネコ，イヌ，カメ

## データの準備

In [ ]:
catnamelist = ['ネコ','イヌ','カメ']
CATALIST=[c0img,c1img,c2img] # c0img:ネコ, c1img:イヌ，c2img:カメ

# 訓練用とテスト用に分割
Xtrain,ytrain,Xtest,ytest,NDATA = make_dataset(CATALIST) 

# データ数の確認
ntrain = len(Xtrain)
ntest  = len(Xtest)
print("カテゴリ数 {}   1カテゴリ当たりのデータ数 {}".format(len(CATALIST),NDATA))
print("訓練用 {} テスト用 {}  　データ総数 {}".format(ntrain,ntest,ntrain+ntest))

# 正解データの確認
print(ytest)

In [ ]:
# 変数の宣言
CLASSES= len(CATALIST)  #  カテゴリ数　ここでは　３　
DATASIZE = SIZE *  SIZE * 3

# Model 1
model1 = Sequential()
model1.add(Dense(CLASSES, activation='softmax', input_shape=(DATASIZE,)))

model = model1
model.compile(loss='sparse_categorical_crossentropy',
              optimizer = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])

# データを 浮動小数点数に変換し、[0,255] → [0,1] に変換
X = (np.array(Xtrain).reshape(len(Xtrain),DATASIZE).astype('float32'))/255
y = np.array(ytrain).astype('float32')
es = EarlyStopping(monitor='loss', patience=5)   #  訓練用データのロスが改善されなくなったら5エポック後に停止
tb_cb = TensorBoard(log_dir='tblog', histogram_freq=1, write_graph=True)
csv_logger = CSVLogger('training.log')
hist = model.fit(X , y,
                 epochs=100,
                 verbose=1,
                 callbacks=[es, csv_logger])

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
acc = hist.history['accuracy']
loss = hist.history['loss']
ax1.plot(range(1, len(loss)+1), loss,color=cm.Set1.colors[0],label='loss')
ax2.plot(range(1, len(acc)+1), acc,color=cm.Set1.colors[1],label='accuracy')
ax1.set_ylabel('Loss')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')
ax1.legend()
ax2.legend()
plt.show()

## 訓練データに対する識別実験結果

In [ ]:
# 訓練データに対する識別結果
TrainError, crossTable = recognitionResult(Xdata=X, ydata=ytrain, catnamelist=catnamelist)
crossTable

In [ ]:
# 誤認識した画像を表示
showEimg(TrainError,Xtrain)
# 0:ネコ、1:イヌ，2:カメ

## テストデータに対する識別実験結果

In [ ]:
# テストデータでの識別結果
X = (np.array(Xtest).reshape(len(Xtest),DATASIZE).astype('float32'))/255
TestError, crossTable = recognitionResult(Xdata=X, ydata=ytest, catnamelist=catnamelist)
crossTable

In [ ]:
# 誤認識した画像を表示
showEimg(TestError,Xtest)
# 0:ネコ、1:イヌ，2：カメ

# model2

In [ ]:
# Model 2

model2 = Sequential()
model2.add(Dense(1024,activation='relu',input_shape=(DATASIZE,)))
# model2.add(Dropout(rate=0.5))
model2.add(Dense(CLASSES, activation='softmax'))

model = model2
model.compile(loss='sparse_categorical_crossentropy',
              optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])

# データを 浮動小数点数に変換し、[0,255] → [0,1] に変換
X = (np.array(Xtrain).reshape(len(Xtrain),DATASIZE).astype('float32'))/255
y = np.array(ytrain).astype('float32')
es = EarlyStopping(monitor='loss', patience=5)   #  訓練用データのロスが改善されなくなったら2エポック後に停止
tb_cb = TensorBoard(log_dir='tblog', histogram_freq=1, write_graph=True)
csv_logger = CSVLogger('training.log')
hist = model.fit(X , y,
                 epochs=100,
                 verbose=1,
                 callbacks=[es, csv_logger])

## 訓練データに対する識別実験結果

In [ ]:
# 訓練データに対する識別結果
TrainError, crossTable = recognitionResult(Xdata=X, ydata=ytrain, catnamelist=catnamelist)
crossTable

In [ ]:
# 誤認識した画像を表示
showEimg(TrainError,Xtrain)
# 0:ネコ、1:イヌ，2:カメ

# テストデータに対する識別実験結果

In [ ]:
# テストデータでの識別結果
X = (np.array(Xtest).reshape(len(Xtest),DATASIZE).astype('float32'))/255
TestError, crossTable = recognitionResult(Xdata=X, ydata=ytest, catnamelist=catnamelist)
crossTable

In [ ]:
# 誤認識した画像を表示
showEimg(TestError,Xtest)
# 0:ネコ、1:イヌ，2：カメ

# model3

In [ ]:
# Model ３

model3 = Sequential()
model3.add(Conv2D(32, kernel_size=(3, 3),input_shape=(SIZE, SIZE,3), activation='relu',padding='same'))
model3.add(Conv2D(64, kernel_size=(3, 3), activation='relu',padding='same'))
model3.add(MaxPooling2D(pool_size=(2, 2)))  # 64 x 64 x 64
model3.add(Conv2D(128, kernel_size=(3, 3), activation='relu',padding='same'))
model3.add(MaxPooling2D(pool_size=(2, 2)))  #  32 x 32 x 128
model3.add(Conv2D(64, kernel_size=(3, 3), activation='relu',padding='same'))
model3.add(Flatten())
model3.add(Dense(1024, activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(64, activation='relu'))
model3.add(Dense(CLASSES, activation='softmax'))

model3.compile(loss='categorical_crossentropy',
              optimizer = optimizers.Adam(),
              metrics=['accuracy'])

model = model3
model.compile(loss='sparse_categorical_crossentropy',
              optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])

# データを 浮動小数点数に変換し、[0,255] → [0,1] に変換
X = (np.array(Xtrain).astype('float32'))/255
y = np.array(ytrain).astype('float32')
es = EarlyStopping(monitor='loss', patience=3)   #  訓練用データのロスが改善されなくなったら3エポック後に停止
tb_cb = TensorBoard(log_dir='tblog', histogram_freq=1, write_graph=True)
csv_logger = CSVLogger('training.log')
hist = model.fit(X , y,
                 epochs=100,
                 verbose=1,
                 callbacks=[es, csv_logger])

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
acc = hist.history['accuracy']
loss = hist.history['loss']
ax1.plot(range(1, len(loss)+1), loss,color=cm.Set1.colors[0],label='loss')
ax2.plot(range(1, len(acc)+1), acc,color=cm.Set1.colors[1],label='accuracy')
ax1.set_ylabel('Loss')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')
ax1.legend()
ax2.legend()
plt.show()

# 訓練データに対する識別実験結果

In [ ]:
# 訓練データに対する識別結果
TrainError, crossTable = recognitionResult(Xdata=X, ydata=ytrain, catnamelist=catnamelist)
crossTable

In [ ]:
# 誤認識した画像を表示
showEimg(TrainError,Xtrain)
# 0:ネコ、1:イヌ，2:カメ

## テストデータに対する識別実験結果

In [ ]:
# テストデータでの識別結果
Xte = (np.array(Xtest).astype('float32'))/255
TestError, crossTable = recognitionResult(Xdata=Xte, ydata=ytest, catnamelist=catnamelist)
crossTable

In [ ]:
# 誤認識した画像を表示
showEimg(TestError,Xtest)
# 0:ネコ、1:イヌ，2：カメ

# 課題

**水増しなしと水増しありの結果を比較しなさい。**


余裕があれば次のような追加実験をしましょう．同じ実験を何度か繰り返し，同じ傾向がみられるかどうか検証すること。


In [ ]:
i